In [1]:
import pandas as pd

In [6]:

df_preventiva = pd.read_excel('Preventiva_08-01-2026.xlsx')
df_carreta = pd.read_excel('CARRETA.xlsx')
df_esl = pd.read_excel('magazine_2026_01_08_08_38.xlsx')
df_mobile = pd.read_excel('Mobile_08-01-2026.xls')
df_bipe = pd.read_excel('Bipe_Produtos_08-01-2026.xlsx')
df_bipe_notas = pd.read_excel('Bipe_de_notas_08-01-2026.xlsx', sheet_name='Plan1')

In [7]:
dfs = [df_preventiva, df_carreta, df_esl , df_mobile,df_bipe, df_bipe_notas  ]

In [8]:
for col in dfs:
    col.columns = col.columns.str.strip()

In [9]:
df_carreta.columns

Index(['PEDIDO', 'NF`s', 'ROMANEIO/ID', 'TRANSP / MOTO', 'CD/FILIAL',
       'DATA_ROMANEIO', 'DATA ENTRADA', 'HORA', 'PREVISÃO ENTREGA', 'TIPO',
       'PEDIDOS SISTEMA', 'CHAVE', 'OBS', 'UUID'],
      dtype='object')

In [10]:
list_colunas = ['PEDIDO', 'NF`s', 'PREVISÃO ENTREGA', 'TIPO', 'CHAVE']

In [11]:
df_carreta[list_colunas]

,PEDIDO,NF`s,PREVISÃO ENTREGA,TIPO,CHAVE
0,1113700712,204382,2025-11-20,Porta a Porta,52251147960950110733550070002043821025366722
1,1111178840,204380,2025-11-20,Porta a Porta,52251147960950110733550070002043801087422567
2,1113088084,204381,2025-11-20,Porta a Porta,52251147960950110733550070002043811021363330
3,1111168149,204379,2025-11-20,Porta a Porta,52251147960950110733550070002043791039116132
4,1111170507,204378,2025-11-20,Porta a Porta,52251147960950110733550070002043781018152388
...,...,...,...,...,...
12179,1129559519,327244,2026-01-08,Porta a Porta,52260147960950110733550070003272445035947751
12180,1129587442,327259,2026-01-08,Porta a Porta,52260147960950110733550070003272595010857972
12181,1129565870,327250,2026-01-08,Porta a Porta,52260147960950110733550070003272505066884023
12182,1129580810,327258,2026-01-08,Porta a Porta,52260147960950110733550070003272585075853273


In [12]:
df_carreta = df_carreta[list_colunas]

In [13]:
df_mobile.columns

Index(['Pedido Cliente', 'Pedido', 'UUID', 'Romaneio', 'Transportador',
       'Nota Fiscal', 'Numero de Série', 'Data Importação Pedido',
       'Data de Captura', 'Data de Processamento', 'Entregador', 'Tipo'],
      dtype='object')

In [11]:
df_bipe_notas.columns

Index(['NF', 'CHAVE', 'OCORRENCIA', 'CD'], dtype='object')

In [12]:
df_carreta.columns

Index(['PEDIDO', 'NF`s', 'PREVISÃO ENTREGA', 'TIPO', 'CHAVE'], dtype='object')

In [15]:
df_final = (pd.merge(df_carreta, df_esl[['Nota Fiscal/Chave NF-e',
                 'Última ocorrência/Observações',
                 'Pessoa/Nome',
                 'Última ocorrência/Data ocorrência']], left_on='CHAVE', right_on='Nota Fiscal/Chave NF-e',
                 how='left').drop(columns=['Nota Fiscal/Chave NF-e'])
                 .merge(df_mobile[['Pedido','Tipo', 'Entregador' ]], left_on='PEDIDO', right_on='Pedido', how='left').drop(columns=['Pedido'])
                 .merge(df_bipe[['PEDIDO_BIPE', 'BIPE_PRODUTO']], left_on='PEDIDO', right_on='PEDIDO_BIPE', how='left').drop(columns=['PEDIDO_BIPE'])
                 .merge(df_bipe_notas[['NF','BIPE_DE_NOTAS']], left_on='NF`s', right_on='NF', how='left').drop(columns=['NF'])
        
                 )

In [16]:
df_final_2 = (pd.merge(df_carreta,df_esl[['Nota Fiscal/Chave NF-e', 'Ocorrência/Ocorrência']], left_on='CHAVE', right_on='Nota Fiscal/Chave NF-e', how='left')
              .drop(columns=['Nota Fiscal/Chave NF-e']))

In [24]:
df_final.head(2)

,PEDIDO,NF`s,PREVISÃO ENTREGA,TIPO,CHAVE,Última ocorrência/Observações,Pessoa/Nome,Última ocorrência/Data ocorrência,Tipo,Entregador,INFO,OCORRENCIA
0,1117092784,237373,2025-12-01,Porta a Porta,52251147960950110733550070002373731034600479,NaN,VALDEVI DOS REIS FRANCISCO (GOIANIA),2025-12-02 23:00:00,Entrega Realizada Normalmente,john Andrew,NaN,NaN
1,1117143808,237379,2025-12-01,Porta a Porta,52251147960950110733550070002373791018718413,NaN,NaN,2025-12-03 20:00:00,Entrega Realizada Normalmente,JHON CLEVER(GOIANIA),NaN,NaN


In [17]:
df_final['Última ocorrência/Observações'] = df_final['Última ocorrência/Observações'].fillna('Não informado')

In [38]:
df_final.columns

Index(['PEDIDO', 'NF`s', 'PREVISÃO ENTREGA', 'TIPO', 'CHAVE',
       'Última ocorrência/Observações', 'Pessoa/Nome',
       'Última ocorrência/Data ocorrência', 'Tipo', 'Entregador', 'INFO',
       'OCORRENCIA'],
      dtype='object')

In [39]:
df_final_2.columns

Index(['PEDIDO', 'NF`s', 'PREVISÃO ENTREGA', 'TIPO', 'CHAVE',
       'Ocorrência/Ocorrência'],
      dtype='object')

In [36]:
df_final.head(2)

,PEDIDO,NF`s,PREVISÃO ENTREGA,TIPO,CHAVE,Última ocorrência/Observações,Pessoa/Nome,Última ocorrência/Data ocorrência,Tipo,Entregador,INFO,OCORRENCIA
0,1117092784,237373,2025-12-01,Porta a Porta,52251147960950110733550070002373731034600479,NaN,VALDEVI DOS REIS FRANCISCO (GOIANIA),2025-12-02 23:00:00,Entrega Realizada Normalmente,john Andrew,NaN,NaN
1,1117143808,237379,2025-12-01,Porta a Porta,52251147960950110733550070002373791018718413,NaN,NaN,2025-12-03 20:00:00,Entrega Realizada Normalmente,JHON CLEVER(GOIANIA),NaN,NaN


In [18]:
df_final_3 = pd.merge(df_final, df_final_2[['CHAVE', 'Ocorrência/Ocorrência']], left_on='CHAVE', right_on='CHAVE', how='left').drop(columns=['CHAVE'])

In [63]:
df_final_3.head(2)

,PEDIDO,NF`s,PREVISÃO ENTREGA,TIPO,Última ocorrência/Observações,Pessoa/Nome,Última ocorrência/Data ocorrência,Tipo,Entregador,INFO,OCORRENCIA,Ocorrência/Ocorrência
0,1117092784,237373,2025-12-01,Porta a Porta,Não informado,VALDEVI DOS REIS FRANCISCO (GOIANIA),2025-12-02 23:00:00,Entrega Realizada Normalmente,john Andrew,NaN,NaN,Entrega Realizada Normalmente
1,1117143808,237379,2025-12-01,Porta a Porta,Não informado,NaN,2025-12-03 20:00:00,Entrega Realizada Normalmente,JHON CLEVER(GOIANIA),NaN,NaN,ENTREGUE


In [19]:
# 1. Cria uma condição (filtro) onde a coluna é 'Não informado'
filtro = df_final_3['Última ocorrência/Observações'] == 'Não informado'

# 2. Nas linhas onde o filtro for verdadeiro, preenche com o valor da outra coluna
df_final_3.loc[filtro, 'Última ocorrência/Observações'] = df_final_3.loc[filtro, 'Ocorrência/Ocorrência']

In [79]:
df_final_3.head(2)

,PEDIDO,NF`s,PREVISÃO ENTREGA,TIPO,Última ocorrência/Observações,Pessoa/Nome,Última ocorrência/Data ocorrência,Tipo,Entregador,INFO,OCORRENCIA,Ocorrência/Ocorrência
0,1117092784,237373,2025-12-01,Porta a Porta,Entrega Realizada Normalmente,VALDEVI DOS REIS FRANCISCO (GOIANIA),2025-12-02 23:00:00,Entrega Realizada Normalmente,john Andrew,NaN,NaN,Entrega Realizada Normalmente
1,1117143808,237379,2025-12-01,Porta a Porta,ENTREGUE,NaN,2025-12-03 20:00:00,Entrega Realizada Normalmente,JHON CLEVER(GOIANIA),NaN,NaN,ENTREGUE


In [80]:
df_final_3.columns

Index(['PEDIDO', 'NF`s', 'PREVISÃO ENTREGA', 'TIPO',
       'Última ocorrência/Observações', 'Pessoa/Nome',
       'Última ocorrência/Data ocorrência', 'Tipo', 'Entregador', 'INFO',
       'OCORRENCIA', 'Ocorrência/Ocorrência'],
      dtype='object')

In [86]:
df_final_3 =  df_final_3.drop(columns=['Ocorrência/Ocorrência'])

In [87]:
df_final_3.head(2)

,PEDIDO,NF`s,PREVISÃO ENTREGA,TIPO,Última ocorrência/Observações,Pessoa/Nome,Última ocorrência/Data ocorrência,Tipo,Entregador,INFO,OCORRENCIA
0,1117092784,237373,2025-12-01,Porta a Porta,Entrega Realizada Normalmente,VALDEVI DOS REIS FRANCISCO (GOIANIA),2025-12-02 23:00:00,Entrega Realizada Normalmente,john Andrew,NaN,NaN
1,1117143808,237379,2025-12-01,Porta a Porta,ENTREGUE,NaN,2025-12-03 20:00:00,Entrega Realizada Normalmente,JHON CLEVER(GOIANIA),NaN,NaN


In [20]:
df_final_3.to_excel('Relatorio.xlsx', sheet_name='Relatorio_01', index=False)

In [8]:
import pandas as pd

In [16]:
import os

caminho_script = os.path.abspath('__file__')

diretorio_raiz = os.path.dirname(caminho_script)

caminho_dado = os.path.join(diretorio_raiz, 'dados')


In [17]:
df_preventiva_2 = None


for arquivo  in os.listdir(caminho_dado):
    caminho_arquivo = os.path.join(caminho_dado, arquivo)

    if 'Preventiva' in arquivo:
        df_preventiva_2 = pd.read_excel(caminho_arquivo)





In [18]:
df_preventiva_2

,cidade_cliente,PEDIDO 1P/FULL
0,APARECIDA DE GOIANIA,1129284676
1,GOIANIA,1129344447
2,GOIANIA,1129240963
3,TRINDADE,1129538494
4,APARECIDA DE GOIANIA,1129507406
...,...,...
92,GOIANIA,1129318012
93,GOIANIA,1129484675
94,APARECIDA DE GOIANIA,1129567590
95,GOIANIA,1129228041
